In [1]:
import sys
sys.path.append('..')
import crystalmaths
from crystalmaths.find_matching_angles import find_matching_angles
import itertools
import requests
import numpy as np
import pandas as pd
%matplotlib qt
# from crystalmaths import 


In [ ]:
image_filepath = r'C:\Users\lacho\OneDrive - UW\crystalmaths\Local_data\F20-atom_res-prev.jpg'

image_object = crystalmaths.imagetools.ImageHandler(image_filepath=image_filepath)

In [ ]:
image_object.get_scale_ratio()
image_object.scale_ratio

In [ ]:
image_object.get_planes()

In [ ]:
image_object.point_coordinates

In [ ]:
#Attempting to shift origin by subtracting center value from points.

p1, p2 = image_object.point_coordinates
print(p1, p2, image_object.image_array.shape[0]/2)
p1[0] = p1[0] - image_object.image_array.shape[0]/2
p1[1] = p1[1] - image_object.image_array.shape[1]/2
p2[0] = p2[0] - image_object.image_array.shape[0]/2
p2[1] = p2[1] - image_object.image_array.shape[1]/2

print(p1, p2)

In [ ]:
fft_angle = crystalmaths.angles_from_fft.angle(p1,p2)
fft_angle

In [ ]:
d1 = crystalmaths.distance_from_fft.dis_cal(p1, image_object.scale_ratio)
d2 = crystalmaths.distance_from_fft.dis_cal(p2, image_object.scale_ratio)

print(d1, d2)

In [2]:
d1 = 3.3
d2 = 3.5
link = crystalmaths.get_d.make_web_address(d1, d2, tolerance=0.001)
link_list = crystalmaths.get_d.compile_links(link)
print(link)

http://rruff.geo.arizona.edu/AMS/result.php?diff=vals(3.3,3.5),opt(),type(d-spacing),tolerance(0.001)


In [10]:
query_results = crystalmaths.get_d.get_d(link_list, [d1, d2])
result_df, metadata_df

In [24]:
def get_zone_axis(image_object, d_spacing_tolerance, angle_tolerance):
    """
    This function will take one image_object which has had two planes
    selected on it. The two points (e.g. d1, d2) will be passed as 
    arguments to get_d, which searches the ACMS online database to
    generate a list of candidate minerals, based on the d-spacing 
    of the selected points from fft image.
    
    
    """
    link = crystalmaths.get_d.make_web_address(d1, d2, tolerance=0.001)
    link_list = crystalmaths.get_d.compile_links(link)
    crystalmaths.get_d.get_d()
    return


for i,result in enumerate(query_results):
    data_df = result[0]
    metadata_df = result[1]
    temp_object = crystalmaths.angles_from_structure_data.AllAnglePairs(data_df, metadata_df)
    new_tuple_list.append(tuple(temp_object.result_df, metadata_df))
# temp_object.result_df.head()



In [26]:
fake_angle = [temp_object.result_df['angle'].mean()]
result_df = temp_object.result_df
result_df = find_matching_angles(fake_angle, result_df, 2)
result_df

,D-REF1,D-REF2,D-SPACING1,D-SPACING2,H1,K1,L1,H2,K2,L2,angle,angle-fft,angle match,zone axis
0,3.3,3.5,3.3955,3.5916,4.0,1.0,0.0,3.0,0.0,1.0,41.699643,40.254734,True,"[1.0, -4.0, -3.0]"
1,3.3,3.5,3.3955,3.5000,4.0,1.0,0.0,4.0,0.0,0.0,14.036243,40.254734,False,None
2,3.3,3.5,3.3955,3.4789,4.0,1.0,0.0,3.0,1.0,1.0,38.418638,40.254734,True,"[1.0, -4.0, 1.0]"
3,3.3,3.5,3.2998,3.5916,3.0,3.0,0.0,3.0,0.0,1.0,57.029685,40.254734,False,None
4,3.3,3.5,3.2998,3.5000,3.0,3.0,0.0,4.0,0.0,0.0,45.000000,40.254734,False,None
5,3.3,3.5,3.2998,3.4789,3.0,3.0,0.0,3.0,1.0,1.0,45.344195,40.254734,False,None


Future Work

# Edwin's code starts from here.

In [ ]:
#This is the function passing all data frame of query results
#Start of all testing and comparison
full_angle_list = []
full_plane1_list = []
full_plane2_list = []
for k in range(len(query_results)):
    plane1_list = []
    plane2_list = []
    df1 = query_results[k][0]
    df11 = df1.loc[df1['D-REF'] == 3.3]
    df12 = df1.loc[df1['D-REF'] == 3.5]
    df_p = query_results[k][1]
    a = []
    lattice_parameter = [float(df_p['a'][0]), float(df_p['b'][0]), float(df_p['c'][0])]
    alpha = float(df_p['alpha'][0])
    beta = float(df_p['beta'][0])
    gamma = float(df_p['gamma'][0])
    for i in range(df11.shape[0]):
        for j in range(df12.shape[0]):
            plane1 = [df11['H'].iloc[i], df11['K'].iloc[i], df11['L'].iloc[i]]
            plane2 = [df12['H'].iloc[j], df12['K'].iloc[j], df12['L'].iloc[j]]
            a.append(crystalmaths.temp_angles_from_structure_data_Edwin.angle_for_all(plane1, plane2, lattice_parameter, alpha, beta, gamma))
            plane1_list.append(plane1)
            plane2_list.append(plane2)
#     print(a)
    full_angle_list.append(a)
    full_plane1_list.append(plane1_list)
    full_plane2_list.append(plane2_list)
    
full_angle_list

In [ ]:
#Given angles and find zone axis
angle_list1 = [11, 19, 34, 42]
tolerance = 1
matching_angle_list = []
for i in range(5):
    angle_list2 = full_angle_list[i]
    c = crystalmaths.find_matching_angles.find_matching_angles(angle_list1, angle_list2, tolerance)
    matching_angle_list.append(c)
    
interested_plane1 = []
interested_plane2 = []
zone_axis_dic = {}
for j in range(len(matching_angle_list)):
    zone_axis = []
    for k in range(len(matching_angle_list[j])):
        for l in range(len(full_angle_list[j])):
            if matching_angle_list[j][k][1] == full_angle_list[j][l]:
                interested_plane1.append(full_plane1_list[j][l])
                interested_plane2.append(full_plane2_list[j][l])
                zone_axis.append(np.cross(full_plane1_list[j][l], full_plane2_list[j][l]))  
                zone_axis_dic[query_results[j][1]['Mineral_Name'][0]] = zone_axis
print(zone_axis_dic)

            
            
